In [1]:
from src.environment import NormalMoveEnv
from src.experiment import Experiment
from src.nn import BetaModel, Model, GeneralModel
from src.rl import Agent, Trajectory

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

%matplotlib agg
# cmd -> cd .\notebooks\
# cmd -> mlflow ui

In [2]:
experiment_name = 'K-Models vs General'
experiment_description = 'Comparação entre uso de n modelos com o uso de um modelo generalista em um mesmo ambiente fixo'
num_runs = 3

In [3]:
import mlflow
from mlflow import MlflowException

try:
    exp_id = mlflow.create_experiment(experiment_name, tags={'mlflow.note.content':experiment_description})
    exp = mlflow.get_experiment(exp_id)
except MlflowException as e:
    exp = mlflow.get_experiment_by_name(experiment_name)
    exp_id = exp.experiment_id

print('experiment_id:', exp_id)

experiment_id: 851126242752723457


In [4]:
env_params= {
    # 'walls': [([-9.,-1.],[10., 1.])],
    'walls': [],
    'goals': [([-9.,8.],[-8.,9.])],
    'start': ([0.,-9.9],[9.9,-5.]),
    'beta': lambda s: (s[1]>-4. and s[1]<4.)
}
model_params = {
    "lr":1e-6, 
    'momentum':.9,
    'td_model_steps': 10, 
    'memory_size': 0, 
    'max_plan_size': 3, 
}
k_model_params = {
    'k':5, 
    'model': Model
}
general_params = {
    'model': GeneralModel
}

In [5]:
def gen_env():
    mlflow.log_params(env_params)
    env = NormalMoveEnv(**env_params)
    mlflow.log_figure(env.plot().get_figure(), f"env.png")
    return env

def run(env, params, episode_size_limit=300):
    mlflow.log_figure(env.plot().get_figure(), f"env.png")
    mlflow.log_params(model_params)
    mlflow.log_params(params)
    a = Agent(env, **params, **model_params)
    a.episode(size_limit=episode_size_limit)
    
    mlflow.log_figure(a.plot(kind='probs'), "probs.png")
    mlflow.log_figure(a.plot(), "params.png")
    for plan_id, plan in enumerate(a.plans):
        mlflow.log_figure(a.plot(kind='plan', plan=plan_id), f"plans/plan-{plan_id:03d}.png")

    mlflow.log_metric("Trajectory Size", a.trajectory.run.shape[0])
    mlflow.log_metric("Final Likelyhood", a.lls[-1])

In [6]:
for i in range(num_runs):
    with mlflow.start_run(experiment_id=exp_id, run_name=f'{i}'):
        env = gen_env()
        with mlflow.start_run(experiment_id=exp_id, run_name='K-Models', nested=True,):
            mlflow.log_param("run", i)
            run(env, k_model_params)
        with mlflow.start_run(experiment_id=exp_id, run_name='General', nested=True,):
            mlflow.log_param("run", i)
            run(env, general_params)

C:\Users\guilherme.albarrans\Workspace\kmbrl\src\rl.py:289: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(5, 5))


Parametros estimados para cada estado.
---ERRO---
self.plans:  123
self.plans[-1]:  1
self.plan_step:  1
Parametros estimados para cada estado.
---ERRO---
self.plans:  80
self.plans[-1]:  0
self.plan_step:  1
---ERRO---
self.plans:  83
self.plans[-1]:  0
self.plan_step:  1
---ERRO---
self.plans:  84
self.plans[-1]:  0
self.plan_step:  1
---ERRO---
self.plans:  85
self.plans[-1]:  0
self.plan_step:  1
Parametros estimados para cada estado.


: 